In [21]:
import pandas as pd

In [49]:
access_data = pd.read_csv('../data/data-clean/food_access_county_data.csv')
environ_data = pd.read_csv('../data/data-clean/food_environ_county_data_clean.csv')

In [51]:
# Pivot the data
pivoted_data = environ_data.pivot_table(
    index=['State', 'County'],       # Rows of the new DataFrame
    columns='Variable_Code',          # Columns of the new DataFrame
    values='Value',                   # Values in the new DataFrame
    aggfunc='first'                   # Function to use for aggregating values if there are duplicates
).reset_index()

# Flatten the column multi-index
pivoted_data.columns.name = None
pivoted_data.columns = [col if isinstance(col, str) else col[1] for col in pivoted_data.columns]

# Print the results
print(pivoted_data.head())

# Optionally, save to a new CSV file
pivoted_data.to_csv('../data/data-clean/food_environ_county_pivoted.csv', index=False)

  State          County  CHILDPOVRATE15  CONVS11  CONVS16  CONVSPTH11  \
0    AK  Aleutians East            15.5      0.0      0.0    0.000000   
1    AK  Aleutians West             8.5      2.0      1.0    0.357974   
2    AK       Anchorage            12.4     60.0     51.0    0.202504   
3    AK          Bethel            31.5      6.0      5.0    0.344511   
4    AK     Bristol Bay             8.6      1.0      0.0    0.970874   

   CONVSPTH16  FFR11  FFR16  FFRPTH11  ...  SPECSPTH11  SPECSPTH16  SUPERC11  \
0    0.000000    1.0    1.0  0.306560  ...    0.000000    0.000000       0.0   
1    0.174004    2.0    0.0  0.357974  ...    0.000000    0.174004       0.0   
2    0.171500  201.0  208.0  0.678387  ...    0.091127    0.067255       9.0   
3    0.278087    7.0    6.0  0.401929  ...    0.000000    0.000000       0.0   
4    0.000000    0.0    0.0  0.000000  ...    0.000000    0.000000       0.0   

   SUPERC16  SUPERCPTH11  SUPERCPTH16  WICS11  WICS16  WICSPTH11  WICSPTH16  
0 

In [57]:
variable_names = pd.read_csv('../data/variable-names/food_environ_variable_list_primary.csv')

# Create a mapping from description to Variable_Code
description_to_code = variable_names.set_index('Variable Name')['Variable Code'].to_dict()

# List of descriptions to keep
descriptions_to_keep = [
    'Grocery stores, 2016',
    'Convenience stores, 2016',
    'Fast-food restaurants, 2016',
    'Farmers\' markets, 2018'
]

# Map descriptions to their codes
codes_to_keep = [description_to_code[desc] for desc in descriptions_to_keep if desc in description_to_code]

# Filter the columns to keep
columns_to_keep = ['State', 'County'] + codes_to_keep
filtered_pivoted_data = pivoted_data[columns_to_keep]
filtered_pivoted_data.columns = ['State', 'County'] + descriptions_to_keep

# Print the results
filtered_pivoted_data.to_csv('../data/data-county-for-processing/food_environ_county_2019.csv')

# geolocation api stuff

In [4]:
filtered_data = nutrition_data[
    (nutrition_data['Year'] == 2019) &
    (nutrition_data['LocationDesc'] == 'Alabama') &
    (nutrition_data['Question'] == 'Percent of adults aged 18 years and older who have obesity')
]

# Display the filtered data
print(filtered_data)

       Unnamed: 0  Year LocationAbbr LocationDesc  \
61912       76504  2019           AL      Alabama   
62647       77315  2019           AL      Alabama   
62740       77420  2019           AL      Alabama   
63465       78217  2019           AL      Alabama   
64104       78925  2019           AL      Alabama   
64421       79277  2019           AL      Alabama   
64492       79351  2019           AL      Alabama   
65153       80065  2019           AL      Alabama   
66412       81452  2019           AL      Alabama   
66454       81498  2019           AL      Alabama   
67184       82308  2019           AL      Alabama   
67547       82711  2019           AL      Alabama   
67601       82769  2019           AL      Alabama   
68071       83294  2019           AL      Alabama   
70889       86414  2019           AL      Alabama   
71177       86729  2019           AL      Alabama   
72352       88019  2019           AL      Alabama   
73030       88767  2019           AL      Alab

In [12]:
# from geopy.geocoders import Photon

# # Create a geolocator object
# geolocator = Photon(user_agent="geoapiExercises")

# # Define the coordinates
# coord = (64.845079957001, -147.722059036)

# # Perform reverse geocoding
# location = geolocator.reverse(coord, exactly_one=True)

# # Extract the county from the location data
# if location:
#     address = location.raw.get('address', {})
#     county = address.get('county', 'County not found')
#     print(f'The coordinates {coord} are in {county} county.')
# else:
#     print('County not found')

from geopy.geocoders import Photon
geolocator = Photon(user_agent="geoapiExercises")
location = geolocator.reverse("34.86597028, -111.763811277")
print(location.address)
print(location.address.split(",")[3])

Red Rock Scenic Byway, 86336, Arizona, United States
 United States


In [1]:
import requests

# full unique geolocations dataset
unique_geolocations = pd.DataFrame(nutrition_data['GeoLocation'].drop_duplicates())
print(unique_geolocations)
# unique_geolocations = unique_geolocations.head(5)
# print(unique_geolocations['GeoLocation'])

# Replace 'YOUR_API_KEY' with your actual Google Maps API key
api_key = 'YOUR_API_KEY'

def get_county(coord):
    # convert coord from string to tuple
    coord = tuple(map(float, coord[1:-1].split(', ')))
    if type(coord) is not tuple:
        return -1
    print("here", coord)
    # URL for the Google Maps Geocoding API
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={coord[0]},{coord[1]}&key={api_key}'
    print(url)
    
    # Send the request
    response = requests.get(url)
    data = response.json()
    
    # Extract the county from the response
    if data['status'] == 'OK':
        for result in data['results']:
            # print(result['address_components'])
            for component in result['address_components']:
                if 'administrative_area_level_2' in component['types']:
                    county = component['long_name']
                    return county
                    # print(f'The coordinates ({latitude}, {longitude}) are in {county}.')
                    # break
    else:
        print('error')
        return -1


# unique_geolocations['County'] = unique_geolocations['GeoLocation'].apply(get_county)

# print(unique_geolocations)

NameError: name 'pd' is not defined